# Neo4j SimpleKGPipeline Integration

This notebook demonstrates the complete Neo4j integration with Vertector, showcasing:
- Document loading and chunking with rich metadata
- Audio transcription and segment-based chunking
- Multimodal pipeline handling both documents and audio
- State isolation between different modalities
- Metadata preservation through the pipeline

## Setup

In [1]:
from pathlib import Path

from vertector_data_ingestion import setup_logging
from vertector_data_ingestion.integrations.neo4j import (
    MultimodalLoader,
    VertectorAudioLoader,
    VertectorDataLoader,
    VertectorTextSplitter,
)

# Setup logging
setup_logging(log_level="INFO")

print("✓ Neo4j integration components imported successfully")

2026-01-09 14:42:48 | INFO     | vertector_data_ingestion.monitoring.logger:setup_logging:51 - Logging initialized at INFO level


✓ Neo4j integration components imported successfully


## Part 1: Document Processing

Load and chunk a PDF document using Docling's structure-aware chunking.

In [3]:
# Initialize document loader and splitter
doc_loader = VertectorDataLoader()
doc_splitter = VertectorTextSplitter(loader=doc_loader, chunk_size=512)

# Load document
pdf_path = Path("../test_documents/sample.pdf")
print(f"Loading document: {pdf_path.name}")

doc_result = await doc_loader.run(pdf_path)

print(f"\n✓ Document loaded:")
print(f"  Type: {doc_result.document_info.document_type}")
print(f"  Pages: {doc_result.document_info.metadata['num_pages']}")
print(f"  Processing time: {doc_result.document_info.metadata['processing_time']}s")
print(f"  Text length: {len(doc_result.text)} characters")

2026-01-09 14:43:41 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2026-01-09 14:43:42 | INFO     | vertector_data_ingestion.core.hardware_detector:detect:50 - Detected Apple Silicon with MPS support
2026-01-09 14:43:42 | INFO     | vertector_data_ingestion.core.pipeline_router:__init__:55 - Hardware detected: mps
2026-01-09 14:43:42 | INFO     | vertector_data_ingestion.core.universal_converter:__init__:44 - Initialized UniversalConverter on mps
2026-01-09 14:43:42 | INFO     | vertector_data_ingestion.core.universal_converter:_ensure_models_available:67 - Checking model availability...
2026-01-09 14:43:42 | INFO     | vertector_data_ingestion.chunkers.hybrid_chunker:__init__:38 - Loading tokenizer: Qwen/Qwen3-Embedding-0.6B (padding_side=left)
2026-01-09 14:43:42 | INFO     | vertector_data_ingestion.chunkers.hybrid_chunker:__init__:50 - Initialized HybridChunker with max_tokens=512, merge_peers=True


Loading document: sample.pdf


2026-01-09 14:43:46 | INFO     | vertector_data_ingestion.core.pipeline_router:determine_pipeline:105 - Using Classic pipeline (default) for sample.pdf
2026-01-09 14:43:46 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:175 - Converting sample.pdf with classic pipeline
2026-01-09 14:43:46,600 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 14:43:46,667 - INFO - Going to convert document batch...
2026-01-09 14:43:46,668 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 870e160bad93d15722a8ae8d62725e09
2026-01-09 14:43:46,698 - INFO - Loading plugin 'docling_defaults'
2026-01-09 14:43:46,700 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-09 14:43:46,712 - INFO - Loading plugin 'docling_defaults'
2026-01-09 14:43:46,718 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']


Consider using the pymupdf_layout package for a greatly improved page layout analysis.


2026-01-09 14:43:47,658 - INFO - Loading plugin 'docling_defaults'
2026-01-09 14:43:47,663 - INFO - Registered layout engines: ['docling_layout_default', 'docling_experimental_table_crops_layout']
2026-01-09 14:43:47,671 - INFO - Accelerator device: 'mps'
2026-01-09 14:43:48,699 - INFO - Loading plugin 'docling_defaults'
2026-01-09 14:43:48,702 - INFO - Registered table structure engines: ['docling_tableformer']
2026-01-09 14:43:51,380 - INFO - Accelerator device: 'mps'
2026-01-09 14:43:52,382 - INFO - Processing document sample.pdf
2026-01-09 14:43:54,055 - INFO - Finished converting document sample.pdf in 7.46 sec.
2026-01-09 14:43:54 | INFO     | vertector_data_ingestion.core.universal_converter:_convert_with_retry:194 - Converted sample.pdf in 10.97s (1 pages, 0.1 pages/sec)



✓ Document loaded:
  Type: document
  Pages: 1
  Processing time: 10.973098039627075s
  Text length: 2713 characters


In [6]:
print(doc_result.text)

## Multi-Domain Balanced Sampling Improves Out-of-Distribution Generalization of Chest X-ray Pathology Prediction Models

Enoch Tetteh Mila, Quebec AI Institute AMMI, AIMS Rwanda etetteh@aimsammi.org

Joseph Viviano Mila, Quebec AI Institute University of

David Krueger Mila, Quebec AI Institute

University of Montreal University of Cambridge

Yoshua Bengio Mila, Quebec AI Institute University of Montreal

Joseph Paul Cohen Mila, Quebec AI Institute AIMI, Stanford University

## Abstract

Learning models that generalize under different distribution shifts in medical imaging has been a long-standing research challenge. There have been several proposals for efficient and robust visual representation learning among vision research practitioners, especially in the sensitive and critical biomedical domain. In this paper, we propose an idea for out-of-distribution generalization of chest X-ray pathologies that uses a simple balanced batch sampling technique. We observed that balanced samplin

In [ ]:
# Chunk the document
print("Chunking document with structure-aware HybridChunker...")
doc_chunks = await doc_splitter.run(doc_result.text)

print(f"\n✓ Created {len(doc_chunks.chunks)} document chunks")
print(f"\nFirst chunk:")
print(f"  Text: {doc_chunks.chunks[0].text[:100]}...")
print(f"  Index: {doc_chunks.chunks[0].index}")
print(f"  Metadata keys: {list(doc_chunks.chunks[0].metadata.keys())}")

In [ ]:
# Examine rich metadata from multiple chunks
print("Document chunk metadata examples:\n")

for i in range(min(3, len(doc_chunks.chunks))):
    chunk = doc_chunks.chunks[i]
    print(f"Chunk {i}:")
    print(f"  Token count: {chunk.metadata.get('token_count')}")
    print(f"  Page number: {chunk.metadata.get('page_no', 'N/A')}")
    print(f"  Section: {chunk.metadata.get('subsection_path', 'N/A')[:80]}")
    print(f"  Is table: {chunk.metadata.get('is_table', 'False')}")
    print(f"  Is heading: {chunk.metadata.get('is_heading', 'False')}")
    print()

## Part 2: Audio Processing

Transcribe and chunk audio using Whisper segments with timestamps.

In [ ]:
# Initialize audio loader and splitter
audio_loader = VertectorAudioLoader()
audio_splitter = VertectorTextSplitter(loader=audio_loader, chunk_size=512)

# Load audio
audio_path = Path("../test_documents/harvard.wav")
print(f"Loading audio: {audio_path.name}")

audio_result = await audio_loader.run(audio_path)

print(f"\n✓ Audio loaded:")
print(f"  Type: {audio_result.document_info.document_type}")
print(f"  Duration: {audio_result.document_info.metadata['duration']}s")
print(f"  Language: {audio_result.document_info.metadata['language']}")
print(f"  Segments: {audio_result.document_info.metadata['segments']}")
print(f"  Model: {audio_result.document_info.metadata['model']}")

In [ ]:
# Chunk the audio
print("Chunking audio using Whisper segments...")
audio_chunks = await audio_splitter.run(audio_result.text)

print(f"\n✓ Created {len(audio_chunks.chunks)} audio chunks")
print(f"\nFirst chunk:")
print(f"  Text: {audio_chunks.chunks[0].text}")
print(f"  Index: {audio_chunks.chunks[0].index}")
print(f"  Metadata keys: {list(audio_chunks.chunks[0].metadata.keys())}")

In [ ]:
# Examine audio chunk metadata with timestamps
print("Audio chunk metadata (with timestamps):\n")

for i, chunk in enumerate(audio_chunks.chunks):
    print(f"Chunk {i}:")
    print(f"  Text: {chunk.text}")
    print(f"  Start: {chunk.metadata['start_time']}s")
    print(f"  End: {chunk.metadata['end_time']}s")
    print(f"  Duration: {chunk.metadata['duration']}s")
    print(f"  Token count: {chunk.metadata['token_count']}")
    print(f"  Document ID: {chunk.metadata['document_id']}")
    print(f"  Chunk ID: {chunk.metadata['chunk_id']}")
    print()

## Part 3: Multimodal Pipeline

Use a single MultimodalLoader to handle both documents and audio, with automatic modality detection.

In [ ]:
# Initialize multimodal loader and splitter
multimodal_loader = MultimodalLoader()
multimodal_splitter = VertectorTextSplitter(loader=multimodal_loader, chunk_size=512)

print("✓ Multimodal pipeline initialized")
print("  Supports: PDF, DOCX, PPTX, XLSX, WAV, MP3, M4A, FLAC, OGG")

### Test 1: Document → Audio (State Isolation)

In [ ]:
# Load and chunk document first
print("Step 1: Processing document...")
doc_result = await multimodal_loader.run(pdf_path)
doc_chunks = await multimodal_splitter.run(doc_result.text)

print(f"✓ Document: {len(doc_chunks.chunks)} chunks")
print(f"  First chunk text: {doc_chunks.chunks[0].text[:80]}...")
print(f"  Has page_no: {'page_no' in doc_chunks.chunks[0].metadata}")
print(f"  Has start_time: {'start_time' in doc_chunks.chunks[0].metadata}")

# Now load and chunk audio (state should be isolated)
print("\nStep 2: Processing audio (after document)...")
audio_result = await multimodal_loader.run(audio_path)
audio_chunks = await multimodal_splitter.run(audio_result.text)

print(f"✓ Audio: {len(audio_chunks.chunks)} chunks")
print(f"  First chunk text: {audio_chunks.chunks[0].text[:80]}...")
print(f"  Has page_no: {'page_no' in audio_chunks.chunks[0].metadata}")
print(f"  Has start_time: {'start_time' in audio_chunks.chunks[0].metadata}")

# Verify state isolation
print("\n✓ State Isolation Verified:")
print(f"  Document chunks != Audio chunks: {doc_chunks.chunks[0].text != audio_chunks.chunks[0].text}")
print(f"  loader.last_document is None: {multimodal_loader.last_document is None}")
print(f"  loader.last_transcription_result is not None: {multimodal_loader.last_transcription_result is not None}")

### Test 2: Audio → Document (Reverse Order)

In [ ]:
# Create fresh loader
fresh_loader = MultimodalLoader()
fresh_splitter = VertectorTextSplitter(loader=fresh_loader, chunk_size=512)

# Load audio first
print("Step 1: Processing audio...")
audio_result = await fresh_loader.run(audio_path)
audio_chunks = await fresh_splitter.run(audio_result.text)

print(f"✓ Audio: {len(audio_chunks.chunks)} chunks")
print(f"  Modality: {audio_chunks.chunks[0].metadata.get('modality', 'N/A')}")

# Then load document
print("\nStep 2: Processing document (after audio)...")
doc_result = await fresh_loader.run(pdf_path)
doc_chunks = await fresh_splitter.run(doc_result.text)

print(f"✓ Document: {len(doc_chunks.chunks)} chunks")
print(f"  Has page_no: {'page_no' in doc_chunks.chunks[0].metadata}")

# Verify state isolation
print("\n✓ Reverse State Isolation Verified:")
print(f"  Audio chunks != Document chunks: {audio_chunks.chunks[0].text != doc_chunks.chunks[0].text}")
print(f"  loader.last_document is not None: {fresh_loader.last_document is not None}")
print(f"  loader.last_transcription_result is None: {fresh_loader.last_transcription_result is None}")

## Part 4: Metadata Comparison

Compare the metadata between document and audio chunks.

In [ ]:
import pandas as pd

# Create comparison DataFrame
comparison = pd.DataFrame({
    'Feature': [
        'chunk_id',
        'document_id', 
        'token_count',
        'page_no',
        'section_title',
        'subsection_path',
        'is_table',
        'is_heading',
        'bbox',
        'modality',
        'start_time',
        'end_time',
        'duration',
        'language'
    ],
    'Document Chunks': [
        '✓' if 'chunk_id' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'document_id' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'token_count' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'page_no' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'section_title' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'subsection_path' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'is_table' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'is_heading' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'bbox' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'modality' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'start_time' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'end_time' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'duration' in doc_chunks.chunks[0].metadata else '✗',
        '✓' if 'language' in doc_chunks.chunks[0].metadata else '✗',
    ],
    'Audio Chunks': [
        '✓' if 'chunk_id' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'document_id' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'token_count' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'page_no' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'section_title' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'subsection_path' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'is_table' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'is_heading' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'bbox' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'modality' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'start_time' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'end_time' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'duration' in audio_chunks.chunks[0].metadata else '✗',
        '✓' if 'language' in audio_chunks.chunks[0].metadata else '✗',
    ]
})

print("Metadata Feature Comparison:")
print(comparison.to_string(index=False))

## Summary

This notebook demonstrated:

✅ **Document Processing**
- Structure-aware chunking with Docling HybridChunker
- Rich metadata: page numbers, sections, bounding boxes, table detection

✅ **Audio Processing**  
- Whisper transcription with MLX acceleration
- Segment-based chunking with timestamps
- Audio metadata: start_time, end_time, duration, language

✅ **Multimodal Pipeline**
- Single loader handles both documents and audio
- Automatic modality detection by file extension
- Proper state isolation between modalities

✅ **Neo4j Integration**
- Compatible with Neo4j SimpleKGPipeline
- Preserves rich metadata for knowledge graph construction
- Ready for production use

### Key Features

1. **Property Delegation**: MultimodalLoader properly exposes sub-loader state
2. **State Isolation**: Loading one modality clears the state of the other
3. **Metadata Preservation**: All rich metadata flows through to Neo4j chunks
4. **Document IDs**: Proper document_id, chunk_id for both documents and audio

### Next Steps

- Connect to Neo4j database
- Define entity and relation schemas
- Build knowledge graph from multimodal data
- Query and visualize the graph